In [1]:
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torch
import pandas as pd

In [2]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")

In [3]:
X = np.load('Data\\X.npy', allow_pickle=True)
y = np.load('Data\\y.npy', allow_pickle=True)

In [4]:
#y_oh = np.zeros((y.shape[0], max(y)+1))
#y_oh[np.arange(y.shape[0]), y] = 1

In [5]:
random.seed(0)
np.random.seed(0)
idx = np.arange(X.shape[0])
random.shuffle(idx)  
btra = np.random.choice(idx, int(0.8*X.shape[0]), replace=False)
btes = [i for i in idx if i not in btra]

In [6]:
Xtra = X[btra]
ytra = y[btra]

Xtes = X[btes]
ytes = y[btes]

In [7]:
Xtes.shape

(195, 571)

In [8]:
def batch_generator(X, Y, n_batches):  
    
    random.seed(123)
    
    batch_size = X.shape[0] // n_batches
    
    idx = np.arange(X.shape[0])
    random.shuffle(idx)    
    idx = idx[:n_batches*batch_size]
        
    for i in range(n_batches):            
        bi = np.random.choice(idx, batch_size, replace=False)
        X_batch = X[bi]
        Y_batch = Y[bi]
        idx = [i for i in idx if i not in bi]
        yield (X_batch,Y_batch)       
        
        
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(571, 24, bias=True)
        self.fc12 = nn.Linear(24, 13, bias=True) 
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)     
        return x1
    
    
def fit(X, Y, Xt, Yt, net, optimizer, error, n_epochs, n_batches, device, PATH, min_val_loss = float('inf')):
    
    net = net.to(device)    
    losses = []    
    val_losses = []

    val_inputs = torch.FloatTensor(Xt)
    val_labels = torch.tensor(Yt, dtype=torch.long)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)  
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0  
         
        
        for batch_x, batch_y in batch_generator(X, Y, n_batches):  
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # get the inputs
            inputs = torch.FloatTensor(batch_x)
            labels = torch.tensor(batch_y, dtype=torch.long)
            inputs, labels = inputs.to(device), labels.to(device)             
                
    
            # forward + backward + optimize
            outputs = net.forward(inputs)
            loss = error(outputs, labels)
                        
            loss.backward()    #obtain gradients      
            optimizer.step()   #optimize
                
            running_loss += loss.item()      
                
        running_loss = running_loss/n_batches    
        
        with torch.no_grad():
            val_outputs = net.forward(val_inputs)
            val_loss = error(val_outputs, val_labels) 
        
        losses.append(running_loss)   
        val_losses.append(val_loss.item())
        
        
        print('Epoch {0}: Training Loss: {1} Validation Loss: {2}'.format(epoch+1, running_loss, val_loss.item()))
        
        if val_loss.item() < min_val_loss:
            torch.save(net.state_dict(), PATH)
            print('New Checkpoint Saved into PATH')
            min_val_loss = val_loss.item()
        
        
def fweights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        #y = 0.0001
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)        
        

In [9]:
n_epochs = 15000
lr = 0.001
n_batches = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'Data\\model_checkpoint.pt'

In [10]:
weights = np.sum(np.unique(ytes, return_counts=True)[1])/np.unique(ytes, return_counts=True)[1]
class_weights = torch.FloatTensor(weights).cuda()

In [11]:
net = Net1()
net.apply(fweights_init_normal)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [12]:
fit(Xtra, ytra, Xtes, ytes, net, optimizer, criterion, n_epochs, n_batches, device, PATH)

Epoch 1: Training Loss: 2.586364984512329 Validation Loss: 2.5788536071777344
New Checkpoint Saved into PATH
Epoch 2: Training Loss: 2.5733602046966553 Validation Loss: 2.5698306560516357
New Checkpoint Saved into PATH
Epoch 3: Training Loss: 2.556111971537272 Validation Loss: 2.563422441482544
New Checkpoint Saved into PATH
Epoch 4: Training Loss: 2.5394643942515054 Validation Loss: 2.5610508918762207
New Checkpoint Saved into PATH
Epoch 5: Training Loss: 2.525557041168213 Validation Loss: 2.5616748332977295
Epoch 6: Training Loss: 2.515100876490275 Validation Loss: 2.5643019676208496
Epoch 7: Training Loss: 2.507951021194458 Validation Loss: 2.567497968673706
Epoch 8: Training Loss: 2.5006144841512046 Validation Loss: 2.5702309608459473
Epoch 9: Training Loss: 2.4945225715637207 Validation Loss: 2.5724704265594482
Epoch 10: Training Loss: 2.492986520131429 Validation Loss: 2.5733482837677
Epoch 11: Training Loss: 2.489573081334432 Validation Loss: 2.5722885131835938
Epoch 12: Trainin

Epoch 82: Training Loss: 2.371384382247925 Validation Loss: 2.4648704528808594
New Checkpoint Saved into PATH
Epoch 83: Training Loss: 2.3667044639587402 Validation Loss: 2.4639039039611816
New Checkpoint Saved into PATH
Epoch 84: Training Loss: 2.3666783968607583 Validation Loss: 2.462888240814209
New Checkpoint Saved into PATH
Epoch 85: Training Loss: 2.3662033081054688 Validation Loss: 2.462228536605835
New Checkpoint Saved into PATH
Epoch 86: Training Loss: 2.364403486251831 Validation Loss: 2.460611343383789
New Checkpoint Saved into PATH
Epoch 87: Training Loss: 2.363484541575114 Validation Loss: 2.4604411125183105
New Checkpoint Saved into PATH
Epoch 88: Training Loss: 2.3618901570638022 Validation Loss: 2.4594244956970215
New Checkpoint Saved into PATH
Epoch 89: Training Loss: 2.360685666402181 Validation Loss: 2.457860231399536
New Checkpoint Saved into PATH
Epoch 90: Training Loss: 2.359774589538574 Validation Loss: 2.4568395614624023
New Checkpoint Saved into PATH
Epoch 91: 

Epoch 158: Training Loss: 2.264084974924723 Validation Loss: 2.378857374191284
New Checkpoint Saved into PATH
Epoch 159: Training Loss: 2.263110955556234 Validation Loss: 2.37784481048584
New Checkpoint Saved into PATH
Epoch 160: Training Loss: 2.262084722518921 Validation Loss: 2.3764636516571045
New Checkpoint Saved into PATH
Epoch 161: Training Loss: 2.260392665863037 Validation Loss: 2.375492572784424
New Checkpoint Saved into PATH
Epoch 162: Training Loss: 2.258196751276652 Validation Loss: 2.374321460723877
New Checkpoint Saved into PATH
Epoch 163: Training Loss: 2.2576348781585693 Validation Loss: 2.373594284057617
New Checkpoint Saved into PATH
Epoch 164: Training Loss: 2.2567657629648843 Validation Loss: 2.3718910217285156
New Checkpoint Saved into PATH
Epoch 165: Training Loss: 2.2540512084960938 Validation Loss: 2.3708715438842773
New Checkpoint Saved into PATH
Epoch 166: Training Loss: 2.2531707286834717 Validation Loss: 2.3704006671905518
New Checkpoint Saved into PATH
Epo

Epoch 234: Training Loss: 2.1534526348114014 Validation Loss: 2.286699056625366
New Checkpoint Saved into PATH
Epoch 235: Training Loss: 2.1508661905924478 Validation Loss: 2.2856452465057373
New Checkpoint Saved into PATH
Epoch 236: Training Loss: 2.15017032623291 Validation Loss: 2.284881830215454
New Checkpoint Saved into PATH
Epoch 237: Training Loss: 2.150364716847738 Validation Loss: 2.2836098670959473
New Checkpoint Saved into PATH
Epoch 238: Training Loss: 2.1498025258382163 Validation Loss: 2.282163143157959
New Checkpoint Saved into PATH
Epoch 239: Training Loss: 2.1468629042307534 Validation Loss: 2.281039237976074
New Checkpoint Saved into PATH
Epoch 240: Training Loss: 2.1445223490397134 Validation Loss: 2.280268907546997
New Checkpoint Saved into PATH
Epoch 241: Training Loss: 2.142122427622477 Validation Loss: 2.279447555541992
New Checkpoint Saved into PATH
Epoch 242: Training Loss: 2.141653140385946 Validation Loss: 2.279113292694092
New Checkpoint Saved into PATH
Epoc

New Checkpoint Saved into PATH
Epoch 310: Training Loss: 2.0389887491861978 Validation Loss: 2.1909663677215576
New Checkpoint Saved into PATH
Epoch 311: Training Loss: 2.0374767780303955 Validation Loss: 2.1899044513702393
New Checkpoint Saved into PATH
Epoch 312: Training Loss: 2.03707488377889 Validation Loss: 2.189770460128784
New Checkpoint Saved into PATH
Epoch 313: Training Loss: 2.034124771753947 Validation Loss: 2.187915325164795
New Checkpoint Saved into PATH
Epoch 314: Training Loss: 2.0324783325195312 Validation Loss: 2.1862964630126953
New Checkpoint Saved into PATH
Epoch 315: Training Loss: 2.03130833307902 Validation Loss: 2.185171365737915
New Checkpoint Saved into PATH
Epoch 316: Training Loss: 2.0329984029134116 Validation Loss: 2.183957576751709
New Checkpoint Saved into PATH
Epoch 317: Training Loss: 2.028276205062866 Validation Loss: 2.1815359592437744
New Checkpoint Saved into PATH
Epoch 318: Training Loss: 2.0267388820648193 Validation Loss: 2.1807894706726074
Ne

...

Epoch 14597: Training Loss: 0.08434999982515971 Validation Loss: 0.7894991040229797
Epoch 14598: Training Loss: 0.08448369552691777 Validation Loss: 0.7890793085098267
Epoch 14599: Training Loss: 0.0846958930293719 Validation Loss: 0.7889817357063293
Epoch 14600: Training Loss: 0.08442703634500504 Validation Loss: 0.7891337871551514
Epoch 14601: Training Loss: 0.08407493680715561 Validation Loss: 0.7889010906219482
Epoch 14602: Training Loss: 0.08456483980019887 Validation Loss: 0.7885016202926636
Epoch 14603: Training Loss: 0.08421353995800018 Validation Loss: 0.7885677218437195
Epoch 14604: Training Loss: 0.08462634434302647 Validation Loss: 0.789057195186615
Epoch 14605: Training Loss: 0.08429107815027237 Validation Loss: 0.7891985177993774
Epoch 14606: Training Loss: 0.08458903680245082 Validation Loss: 0.7899319529533386
Epoch 14607: Training Loss: 0.08435595780611038 Validation Loss: 0.7898470163345337
Epoch 14608: Training Loss: 0.08421069631973903 Validation Loss: 0.78985989093

Epoch 14697: Training Loss: 0.08401954174041748 Validation Loss: 0.7898522615432739
Epoch 14698: Training Loss: 0.08436873306830724 Validation Loss: 0.7894845008850098
Epoch 14699: Training Loss: 0.084341694911321 Validation Loss: 0.7899808287620544
Epoch 14700: Training Loss: 0.0846173365910848 Validation Loss: 0.790522575378418
Epoch 14701: Training Loss: 0.08423776427904765 Validation Loss: 0.7907114028930664
Epoch 14702: Training Loss: 0.0842142105102539 Validation Loss: 0.7911213040351868
Epoch 14703: Training Loss: 0.08414179335037868 Validation Loss: 0.7904083132743835
Epoch 14704: Training Loss: 0.083961288134257 Validation Loss: 0.7893303632736206
Epoch 14705: Training Loss: 0.08397355924050014 Validation Loss: 0.7896798849105835
Epoch 14706: Training Loss: 0.08464345832665761 Validation Loss: 0.7894500494003296
Epoch 14707: Training Loss: 0.08414241919914882 Validation Loss: 0.7901519536972046
Epoch 14708: Training Loss: 0.08413832137982051 Validation Loss: 0.7908037304878235

Epoch 14794: Training Loss: 0.08401954670747121 Validation Loss: 0.7909708023071289
Epoch 14795: Training Loss: 0.08427160481611888 Validation Loss: 0.7912390232086182
Epoch 14796: Training Loss: 0.08378654470046361 Validation Loss: 0.7912914752960205
Epoch 14797: Training Loss: 0.08396054804325104 Validation Loss: 0.7907359600067139
Epoch 14798: Training Loss: 0.08377251774072647 Validation Loss: 0.7904557585716248
Epoch 14799: Training Loss: 0.08373460918664932 Validation Loss: 0.7907143235206604
Epoch 14800: Training Loss: 0.08381718148787816 Validation Loss: 0.7913063168525696
Epoch 14801: Training Loss: 0.08374432474374771 Validation Loss: 0.7913651466369629
Epoch 14802: Training Loss: 0.08488442003726959 Validation Loss: 0.7918769121170044
Epoch 14803: Training Loss: 0.08497470368941624 Validation Loss: 0.7917439341545105
Epoch 14804: Training Loss: 0.08391811698675156 Validation Loss: 0.7910709381103516
Epoch 14805: Training Loss: 0.08417130510012309 Validation Loss: 0.790238082

Epoch 14893: Training Loss: 0.08365948001543681 Validation Loss: 0.7915828824043274
Epoch 14894: Training Loss: 0.08346207688252132 Validation Loss: 0.7915189266204834
Epoch 14895: Training Loss: 0.08338585495948792 Validation Loss: 0.7919602394104004
Epoch 14896: Training Loss: 0.08402787148952484 Validation Loss: 0.7922428846359253
Epoch 14897: Training Loss: 0.08359065403540929 Validation Loss: 0.7923917174339294
Epoch 14898: Training Loss: 0.083360160390536 Validation Loss: 0.7922154068946838
Epoch 14899: Training Loss: 0.08362385630607605 Validation Loss: 0.7920441031455994
Epoch 14900: Training Loss: 0.08435828735431035 Validation Loss: 0.7913820743560791
Epoch 14901: Training Loss: 0.08314357449611028 Validation Loss: 0.7917908430099487
Epoch 14902: Training Loss: 0.08440675338109334 Validation Loss: 0.7915835976600647
Epoch 14903: Training Loss: 0.08353390793005626 Validation Loss: 0.7918387055397034
Epoch 14904: Training Loss: 0.08344022184610367 Validation Loss: 0.79216140508

Epoch 14996: Training Loss: 0.08362149198849995 Validation Loss: 0.7924947738647461
Epoch 14997: Training Loss: 0.08338791131973267 Validation Loss: 0.7928057909011841
Epoch 14998: Training Loss: 0.08330820004145305 Validation Loss: 0.793096125125885
Epoch 14999: Training Loss: 0.08338627715905507 Validation Loss: 0.7927207946777344
Epoch 15000: Training Loss: 0.08314133435487747 Validation Loss: 0.7923822999000549


In [13]:
PATH = 'Data\\model_checkpoint.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [14]:
error = nn.CrossEntropyLoss()

In [15]:
with torch.no_grad():
    net.eval()
    inputs = torch.FloatTensor(Xtes)
    labels = torch.tensor(ytes, dtype=torch.long)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net.forward(inputs)
    loss = error(outputs, labels) 
    print(loss.item())

0.700065016746521


In [16]:
0.700065016746521

0.700065016746521

In [17]:
param_dict = {}
for v,p in enumerate(net.parameters()):
    param_dict[v] = p.data.numpy()
    
W0 = param_dict[0].T
b0 = param_dict[1]
W1 = param_dict[2].T
b1 = param_dict[3]


h0 = np.matmul(Xtes, W0) + b0
h1 = np.tanh(h0)
h2 = np.matmul(h1, W1) + b1
h3 = np.exp(h2)
o = h3/np.sum(h3,axis=1).reshape(-1,1)

In [18]:
o.shape

(195, 13)

In [19]:
np.save('Data\\param_dict.npy', param_dict)

In [20]:
def y_hat(prob, topics):    
    return topics[np.argmax(prob)]
    

In [21]:
pred_labels = []
topics = ['Jubilacion Patronal', 'Consultoria', 'Renuncia/Despido/Desahucio', 'IESS', 
                 'Greeting', 'Contacto', 'No Topic', 'Queja', 'Otros servicios', 'Charlas/Capacitaciones', 
                      'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']
for c in o:
    pred_labels.append(y_hat(c, topics))

In [22]:
preguntas = data.Pregunta[btes]
labels = data.Tema[btes]


In [23]:
Xtes = np.column_stack((preguntas, labels, pred_labels, Xtes, o))


In [24]:
data2 = pd.DataFrame(Xtes)


In [25]:
vocab = np.load('Data\\vocab.npy', allow_pickle=True)
colnames = np.concatenate((['pregunta', 'label', 'pred_label'], vocab, ['n_token', 'perc_greet', 'polarity'], ['Jubilacion Patronal', 'Consultoria', \
                                                                                                     'Renuncia/Despido/Desahucio', 'IESS', 'Greeting', \
                                                                                                         'Contacto', 'No Topic', 'Queja', 'Otros servicios', \
                                                                                                 'Charlas/Capacitaciones', 'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']))
data2.columns = colnames


In [26]:
data2.head(3)


,pregunta,label,pred_label,buen,dia,mi,nombr,llam,empres,me,...,IESS,Greeting,Contacto,No Topic,Queja,Otros servicios,Charlas/Capacitaciones,Hi Five,job seeker,Facturacion/Retencion/Cobros
0,Hay posiciones abiertas en ACTUARIA?,job seeker,Contacto,0,0,0,0,0,0,0,...,0.0117327,0.056362,0.402604,0.068683,0.0550525,0.01466,0.0308319,0.123469,0.129792,0.0181059
1,necesito servicios de capacitacion para obtene...,Charlas/Capacitaciones,Jubilacion Patronal,0,0,0,0,0,0,0,...,0.0101813,0.00103417,0.00517052,0.197485,0.0532342,0.00110557,0.109853,0.00744999,0.000334574,0.00783009
2,Han trabajado con Modelos de Deserción Cliente...,Consultoria,Consultoria,0,0,0,0,0,0,0,...,0.00177936,0.000785761,0.0314653,0.0114919,0.0369084,0.0104964,0.0119156,0.000759216,0.123256,0.0131245


In [27]:
data2.to_csv('Data\\Xtes.csv', sep=',')